Importanje svega

In [70]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader


Nesto za device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Podaci i izbacivanje misssing values

In [23]:
data = pd.read_csv('..\Know center zadatak za vjezbu\df_nno2_new.csv', )
data = data.dropna(axis=0)

features=['Nord_Precip','Nord_Pressure','Nord_RH', 'Nord_Temp', 'Nord_Winddirection', 'Nord_Windspeed', 'weekday_Monday',
       'weekday_Tuesday', 'weekday_Wednesday', 'weekday_Thursday',
       'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday', 'season_fall',
       'season_spring', 'season_summer', 'season_winter', 'holiday_sunday',
       'holiday', 'holiday_school', 'year', 'month', 'dayofyear', 'weekday',
       'season']
X=data[features]
y=data['N_NO2']

print(X)

X1 = X.dropna(axis = 0, how ='any')
print("Old data frame length:", len(X), "\nNew data frame length:", 
       len(X1), "\nNumber of rows with at least 1 NA value: ",
       (len(X)-len(X1)))
X2=X1.loc[(X!=0).any(axis=1)]


y1 = y.dropna(axis = 0, how ='any')
print("Old data frame length:", len(y), "\nNew data frame length:", 
       len(y1), "\nNumber of rows with at least 1 NA value: ",
       (len(y)-len(y1)))



      Nord_Precip  Nord_Pressure    Nord_RH  Nord_Temp  Nord_Winddirection  \
0             0.0      953.79370  77.251884   4.035931            89.42938   
1             0.0      950.12506  81.564156   3.522331           110.14704   
2             0.0      961.61456  56.786457   3.109104           227.58770   
3             0.0      974.15820  49.110626  -0.742807           242.49976   
4             0.0      972.75420  69.331055  -3.867371           134.32390   
...           ...            ...        ...        ...                 ...   
4201          0.0      977.16940  67.474310  18.020380           167.51172   
4202          0.0      976.34150  58.300950  21.290035           150.60567   
4203          0.0      973.71210  68.143060  19.866663           107.73452   
4204          0.0      973.59030  66.033400  21.587740           120.35903   
4205          0.0      974.58840  63.282795  24.060555           104.95655   

      Nord_Windspeed  weekday_Monday  weekday_Tuesday  weekday_

Podjela podataka i scaliranje 

In [64]:
sc= MinMaxScaler()

# podjeliti u trening i test set

X_train=X2[0:3655]
X_test=X2[3656:4207]
Y_train=y1[0:3655]
Y_test=y1[3656:4207]

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Y_train=Y_train.to_numpy()
Y_test=Y_test.to_numpy()


X_train=torch.from_numpy(X_train)
Y_train=torch.from_numpy(Y_train)
X_test=torch.from_numpy(X_test)
Y_test=torch.from_numpy(Y_test)

print(X_test)
print(X_train)
print(Y_test)
print(X_train)


tensor([[0.0000, 0.7444, 0.5597,  ..., 0.0110, 0.0000, 0.0000],
        [0.0019, 0.8550, 0.3892,  ..., 0.0137, 0.1667, 0.0000],
        [0.0000, 0.8801, 0.6144,  ..., 0.0164, 0.3333, 0.0000],
        ...,
        [0.0000, 0.5805, 0.4721,  ..., 0.5068, 0.1667, 0.6667],
        [0.0000, 0.5784, 0.4342,  ..., 0.5096, 0.3333, 0.6667],
        [0.0000, 0.5956, 0.3848,  ..., 0.5123, 0.5000, 0.6667]],
       dtype=torch.float64)
tensor([[0.0000, 0.2373, 0.6356,  ..., 0.0000, 0.8333, 0.0000],
        [0.0000, 0.1741, 0.7130,  ..., 0.0027, 1.0000, 0.0000],
        [0.0000, 0.3720, 0.2682,  ..., 0.0055, 0.0000, 0.0000],
        ...,
        [0.0000, 0.8336, 0.5040,  ..., 0.0000, 0.5000, 0.0000],
        [0.0000, 0.9158, 0.7576,  ..., 0.0027, 0.6667, 0.0000],
        [0.0000, 0.8679, 0.5315,  ..., 0.0055, 0.8333, 0.0000]],
       dtype=torch.float64)
tensor([35.7473, 13.9989, 30.5436, 44.9537, 44.5950, 49.6505, 42.4775, 31.0920,
        31.4828, 46.6792, 43.1284, 45.5344, 53.3597, 36.5592, 30.504

In [78]:
import torch.nn as nn
import torch.nn.functional as F

def swish(X_train):
    return X_train * F.sigmoid(X_train)

class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(25, 16)
        self.b1 = nn.ReLU(16)
        self.fc2 = nn.Linear(16, 8)
        self.b2 = nn.ReLU(8)
        self.fc3 = nn.Linear(8,4)
        self.b3 = nn.ReLU(4)
        self.fc4 = nn.Linear(4,1)

    def forward(self,X_train):

        x = swish(self.fc1(x))
        x = self.b1(x)
        x = swish(self.fc2(x))
        x = self.b2(x)
        x = swish(self.fc3(x))
        x = self.b3(x)
        x = F.sigmoid(self.fc4(x))

        return x

In [77]:
n_iters = 100
learning_rate = 0.001

loss = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)


for epoch in range(n_iters):
    
    # predict = forward pass with our model
    y_predicted = net(X_train)

    # loss
    l = loss(Y_test, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w, b] = net.parameters() # unpack parameters
        print('epoch ', epoch+1, ': w = ', w[0][0].item(), ' loss = ', l)

print(f'Prediction after training: f(5) = {net(X_test).item():.3f}')

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataLoader